In [ ]:
import os
import cv2
import requests
from os import path
import numpy as np
from tqdm import tqdm
from PIL import Image
import pickle
from rectangle import Rectangle
import helpers
from helpers import *
import time
import pickle

kindle_ar = (1430,1080)

if not path.exists(f'dump'):
    os.mkdir('dump')

manga_name = 'Onepunch-Man'

In [ ]:
os.environ["PATH"] += ":"+os.getcwd()
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)
driver.get("https://mangasee123.com/manga/"+manga_name)
# with open('mangasee.html', 'w', encoding='utf-8') as f:
#     f.write(driver.page_source)
selector = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.ShowAllChapters')))
selector.click()

chapter_links = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, 'a[href*="read-online"]')]
chapter_links = reversed(chapter_links)
pages = {}
for chapter in chapter_links:
    driver.get(chapter)

    selector = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'i.fa-arrows-alt-v')))
    selector.click()

    img_links = [img.get_attribute('src') for img in driver.find_elements(By.CSS_SELECTOR, 'img.img-fluid')]
    pages[chapter.split('/')[-1]] = img_links
with open(manga_name+'.pkl', 'wb') as f:
    pickle.dump(pages, f)

In [ ]:
def process_page(url, chapter_counter, page_counter):
    # print(url)
    try:
        raw_img = requests.get(url, stream=True).raw
        img = np.array(Image.open(raw_img))
        if img.ndim==2:
            img = np.repeat(img[:, :, np.newaxis], 3, axis=2)
    except:
        return

    # if not path.exists(f'dump/{manga_name}/{chapter_counter:04d}'):
    #     os.mkdir(f'dump/{manga_name}/{chapter_counter:04d}')
    # img = cv2.imread('0012-002.png')
    # plot_crops(img, [])
    contours = get_contours(img)
    contours = remove_specks_and_lines(img, contours)
    crops = get_crops(img, contours)
    crops = merge_overlapping_crops(crops)
    crops = remove_small_crops(img, crops)
    helpers.debug = False

    text_crops = get_text_crops(img)
    text_crops = remove_small_crops(img, text_crops, factor=0.5)

    crops.extend(text_crops)
    crops = merge_overlapping_crops(crops)

    # debug = True
    # plot_frames(img, crops, True)
    edge_graph = build_crops_edge_graph(crops, img.shape)
    # print(*[(i,j) for i,j in enumerate(edge_graph)], sep='\n')
    sorted_crops = sort_edge_graph(crops, edge_graph)
    idx = 0
    if len(sorted_crops)==1:
        sorted_crops = [Rectangle(0,0,img.shape[1],img.shape[0])]

#         crop_area = sum([c.area for c in sorted_crops])
#         total_area = img.shape[0]*img.shape[1]

#         if crop_area/total_area < 0.80:
#             sorted_crops = [Rectangle(0,0,img.shape[1],img.shape[0])]

    image = plot_frames(img, sorted_crops, True, show=helpers.debug)
    for c in sorted_crops:
        gray_crop = cv2.cvtColor(img[c.y:c.z, c.x:c.w], cv2.COLOR_BGR2GRAY)

        if gray_crop.shape[1]/gray_crop.shape[0]>0.80 or c.area < img.shape[0]*img.shape[1]*0.5:
            gray_crop = [np.rot90(gray_crop)]
        else:
            text_crops = merge_vertical_overlapping_crops(text_crops)
            if len(text_crops)==0:
                middleh = c.height//2+c.y
            elif len(text_crops)==1:
                middleh = c.height//2+c.y
                if text_crops[0].y<=middleh and middleh<=text_crops[0].z:
                    middleh = text_crops[0].y if text_crops[0].y-middleh < middleh-text_crops[0].z else text_crops[0].z
            else: # find the widest gat
                def pairwise(iterable):
                    a = iter(iterable)
                    return zip(a, a)
                widest_gap = max([(idx,j.y-i.z) for idx,(i,j) in enumerate(list(pairwise(text_crops)))], key=lambda t: t[1])
                middleh = text_crops[widest_gap[0]].z+widest_gap[1]//2
            # cv2.line(image, (c.x, middleh), (c.w, middleh), (0, 255, 0), thickness=3)
            gray_crop = [np.rot90(h) for h in np.split(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), [middleh])]

        if helpers.debug:
            plot_crops(image,[])
        # cv2.imwrite(f"dump/{manga_name}/{chapter_counter:04d}/{page_counter:03d}.jpg", image, [int(cv2.IMWRITE_JPEG_QUALITY), 20])
        for gc in gray_crop:
            gc = resizeAndPad(gc, kindle_ar, 255)
            cv2.imwrite(f"dump/{manga_name}/{chapter_counter:04d}-{page_counter:03d}-{idx:03d}.jpg", gc, [int(cv2.IMWRITE_JPEG_QUALITY), 20])
            idx+=1


In [ ]:
if not path.exists(f'dump/{manga_name}'):
    os.mkdir(f'dump/{manga_name}')

with open(manga_name+'.pkl', 'rb') as f:
    pages = pickle.load(f)

chapter_counter = 0
page_counter = 5
chap = [*pages.keys()][chapter_counter]
url = pages[chap][page_counter]
helpers.debug = False
process_page(url, chapter_counter, page_counter)

In [ ]:
from multiprocessing import Pool, cpu_count
pool = Pool(processes=(cpu_count() - 1))

if not path.exists(f'dump/{manga_name}'):
    os.mkdir(f'dump/{manga_name}')

with open(manga_name+'.pkl', 'rb') as f:
    pages = pickle.load(f)

for chapter_counter,chap in enumerate(pages.keys()):
    # if chapter_counter!=3: continue
    for page_counter, url in enumerate(tqdm(pages[chap], desc=f'Chapter {chapter_counter}: {chap}')):
        # if 'Onepunch-Man-chapter-63-page-1.html' not in chap: continue
        # if page_counter!=23: continue
        # print(i)
        pool.apply_async(process_page, args=(url, chapter_counter, page_counter))

pool.close()
pool.join()

In [ ]:
import os
os.system(f"cd dump; zip -FSr {manga_name} {manga_name} -x '*/\.*'")